In [1]:
import cadquery as cq

In [211]:
from jupyter_cadquery.cadquery import (Assembly, Part, Edges, Faces, Vertices, show)
from jupyter_cadquery import set_sidecar

set_sidecar("CadQuery")  # force usage of one cad view on the right

result = (cq.Workplane("XZ").circle(1.5)
    .workplane(offset=3.0).rect(0.75, 0.5).loft(combine=True))

s = cq.Workplane("XZ")
sPnts = [
    (2.75, 1.5),
    (2.5, 1.75),
    (2.0, 1.5),
    (1.5, 1.0),
    (1.0, 1.25),
    (0.5, 1.0),
    (0, 1.0)
]
r = s.lineTo(3.0, 0).lineTo(3.0, 1.0).spline(sPnts,includeCurrent=True).close()
result = r.workplane(offset=3.0).rect(0.75, 0.5) # .loft(combine=True)
result = result.workplane(offset=3.0).circle(1.5).loft(combine=True)
#    result = r.extrude(0.5)
#show_object(result)
#show(result)
show(result, axes=True, grid=True, ortho=True, axes0=True)


Done, using side car 'CadQuery'


In [212]:
def aerospline(r):
    r = r.spline([(0,0), (-1,1), (-2,0.5), (-3,0)], tangents=[(0,1), (-1,-0.3)], includeCurrent=False)
    r = r.spline([(-3,0), (-1,0.1), (0,0)], tangents=[(1,0.1), (0,1)], includeCurrent=False)
    r = r.close()
    return r

def aerosplineD(r, hfac, wfac, xd):
    r = r.spline([(0+xd,0), (-1*wfac+xd,1*hfac), (-2*wfac+xd,0.5*hfac), (-3*wfac+xd,0)], includeCurrent=False)
    r = r.spline([(-3*wfac+xd,0), (-1*wfac+xd,0.1*hfac), (0+xd,0)], includeCurrent=False)
    r = r.close()
    return r

#r = aerosplineD(s, 0.9, 1.8, -4)
#result = r.extrude(0.5)

r = cq.Workplane("XZ", (0,0,0))
r = aerosplineD(r, 1, 1, 0)
r = r.workplane(offset=1, centerOption="ProjectedOrigin")
r = aerosplineD(r, 0.8, 0.9, -0.5)
r = r.workplane(offset=1, centerOption="ProjectedOrigin")
r = aerosplineD(r, 0.5, 0.9, -1.2)
r = r.workplane(offset=2, centerOption="ProjectedOrigin")
r = aerosplineD(r, 0.2, 0.7, -3.6)
r = r.loft(combine=True)

m = r.mirror(mirrorPlane='XZ', basePointVector=(0,0,0))
r = m.union(r)

show(r, axes=True, grid=True, ortho=True, axes0=True)


Done, using side car 'CadQuery'


In [210]:
with open("cqtest.step", "w") as fout:
    cq.exporters.exportShape(r, cq.exporters.ExportTypes.STEP, fout)

# Load this file into Gmsh
# Click 1D, 2D, 3D to mesh edges, surfaces and volumes
# To tighten the mesh, go to Options -> Mesh -> General; Element size factor: 0.5
# You need to download and run the latest version of gmsh
# ~/executables/gmsh-4.5.4-Linux64/bin/gmsh -clscale 0.5 cqtest.step 
#  see https://gitlab.onelab.info/gmsh/gmsh/issues/775



In [ ]:
# Experimenting with sparselizard:

# clone and build sparselizard:
# > git clone git@github.com:halbux/sparselizard.git
# > cd sparselizard
# > ./install_external_libs/install_petsc.sh
# > ./install_external_libs/optional_install_gmsh_api.sh
# > make
# > ./run_sparselizard.sh

# Download and run the visualizer:
# Unzip and run with:
# ~/executables/ParaView-5.8.0-MPI-Linux-Python3.7-64bit/bin/paraview
# Open the file u.vtk.  Enable the view, turn on the colours.

# Now try with air simulation
# > cp examples/nonlinear-vonkarman-vortex-2d/* .
# > gmsh channel.geo
# > Click 3 in mesh, and save mesh
# > make
# > ./run_sparselizard.sh
# (makes a 90 second simulation)

# Future task: 
# channel.geo is in a simplified CAD language.  
# Program this with the aerofoil shape of upright
# Test with and without turbulator

